# Тренируем большую модель на анекдотах

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import numpy as np

In [25]:
with open("../anecdots.txt", "r") as f_in:
    book = f_in.read()
print(book[:500])    


??????????????????????????????? 

?О АВТОМАШИНАХ И ИХ ВОДИТЕЛЯХ ? 

??????????????????????????????? 

Несмотря на красный свет светофора, Луиза пересекла 

перекресток на своей машине и была остановлена полицейским. 

- Мадам, разве вы не видели красного света? - интересуется он. 

- Простите, мосье регулировщик, - отвечает она. - Красный 

свет я видела, а вот вас не заметила. 

Женщина-водитель после столкновения с другой машиной говорит: 

- Это моя вина! 

- Нет, мадам, - галантно отвечает 


# Словарь и токенайзер

In [11]:
vocab = sorted(list(set("".join(book))), key=lambda v: "\t" if v == "." else v)
vocab_size = len(vocab)

In [12]:
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = {index: char for char, index in char_to_index.items()}

def tokenize(char):
    return char_to_index.get(char, 0) 

def untokenize(index):
    return index_to_char.get(index, " ")

# Готовим данные для обучения

In [14]:
data = torch.tensor([tokenize(x) for x in book], dtype=torch.long)
print(data, data.shape)

tensor([ 1, 29, 29,  ...,  3,  2,  1]) torch.Size([914848])


In [15]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
def get_batch(split, batch_size = 4):
    data = val_data if split == "valid" else train_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in idx])
    Y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return(X,Y)

# Функции полезные

In [18]:
@torch.no_grad()
def evaluate_model(model, neval = 500):
    model.eval()
    scores = {}
    for split in ['train', 'valid']:
        loss = 0
        for i in range(neval):
            X, Y = get_batch(split, batch_size=batch_size)
            _, loss_i = model(X, Y)
            loss += loss_i.item()
        scores[split] = loss / neval
    model.train()
    return scores

# Константы

In [44]:
block_size = 256
batch_size = 64
dropout = 0.2
n_embd = 384
dropout = 0.2
n_layer = 6
n_head = 6
device = 'cuda'
n_iter = 5000
learning_rate = 3e-4

# Модель

In [33]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [34]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [45]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [46]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [47]:
class Model7(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Тренировка

In [38]:
model7 = Model7()
optimizer7 = torch.optim.AdamW(model7.parameters(), lr=learning_rate)

In [48]:
m = model7
o = optimizer7
for i in range(n_iter):
    X,Y = get_batch('train', batch_size=batch_size)
    logits, loss = m(X, Y)
    o.zero_grad(set_to_none=True)
    loss.backward()
    o.step()
    if i%1000 == 0:
        scores = evaluate_model(m)
        print(f"Loss train: {scores['train']:.4f}, valid {scores['valid']:.4f}")
print(loss.item())

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [42]:
max_tokens = 1000
prompt = torch.zeros([1, 1], dtype = torch.long)
print("".join([untokenize(x) for x in m.generate(prompt, max_tokens).tolist()[0]]))

. 

А Двад педеет тей прудодашаецт 

ве. я мюне ветецу9ь? 

- Ва. Наютцише темнь дойси вл сссплас, ет то голикеня влолфем.. 


`атрднит яговат цедико! олу: 

- иломомы вазане

- арт сту лоп$осоририн, ка былоно не дно де мубкана дить! - ноболиц к зСализы н вия 

-то Этогоскак трана. 


- Кирей.... А 


Раталь вождтане зилсью мает ирит три х го тсянашаны, чибер 

- баде? -пажит ! M 

- Геротасъевнь моетнодны, даслудинтсь бога сте доми сы тек?. 

- у дня рорулакт Мэтоко всенят стол мавабое ил и стенкол, полил, ту даме, нерол нае стичнлсодо - попрыж, вашеся ваке илокабел напрет. Ро Дасвамусть, я бро наживыг дризика рурнего аму педолавилал аю тогыпа ик и хосты готопромит, сэче 

добядатькриеня питеть восови- Шнонкув крудак окернаю наре-е! 

С? 1оне. 

- Вилени бищона чване шив иносо сяше Вый нит важоринт! ? Си сютска скук когали пол гю, дваховеб. Мь, нтвлиолевку. 
Са. 

- Дашрвохана, пи 

Жинилепе нужищ иена веннодотатилы чукомн оерворудийвосо те" 

Ны кан. со 

- (у Гтаж звомачзжиши метамы